Scratch work and preliminary performance notes for `swe-python` with TT/QTT.

In [1]:
import os
import time
import numpy as np
import netCDF4 as nc
import argparse

from stb import strtobool

from msh import load_mesh, load_flow, \
                sort_mesh, sort_flow
from ops import trsk_mats

from _dx import HH_TINY, UU_TINY
from _dx import invariant, diag_vars, tcpu
from _dt import step_eqns

import torch as tn
import torchtt as tt

from sympy.ntheory import factorint

from temp_init import init_file
from timer import Timer

Here, we make a "fake" `cnfg` object with defaults since we don't have access to `argparse` from the notebook.

In [2]:
class base: pass

cnfg = base()

cnfg.save_freq = 100
cnfg.stat_freq = 100

cnfg.integrate = 'RK44'
cnfg.operators = 'TRSK-CV'
cnfg.equations = 'SHALLOW-WATER'
cnfg.ke_upwind = 'AUST-CONST'
cnfg.ke_scheme = 'CENTRE'
cnfg.pv_upwind = 'AUST-ADAPT'
cnfg.pv_scheme = 'UPWIND'

cnfg.du_damp_4 = 0.0
cnfg.vu_damp_4 = 0.0

cnfg.iteration = 0
cnfg.no_rotate = False

# testing with simple, quasi-linear
# test case on low resolution mesh
name = 'io/ltc1_cvt_5.nc'
path, file = os.path.split(name)
save = os.path.join(path, "out_" + file)

Load configs, init file, and build the RHS TRiSK operators.

The `flow` object stores the current model state, and the `mesh` object stores the mesh itself.

The RHS operators are all stored in the `trsk` object -- these are built as `scipy.sparse.csr_matrix` objects (compressed sparse row matricies). Note that we often see lines of the form
```
trsk.<operator> * <state_vector>
```
This performs a matrix-vector multiply, using the soon-to-be-depreciated `*` symbol, overloaded with the proper sparse matrix multiplication routine.
The modern standard is to use `@` as in `numpy` -- at present, both `*` and `@` do the same thing, with `@` to replace `*` in the next version of `scipy`.

In [3]:
print("Loading input assets...")

# load mesh + init. conditions
mesh = load_mesh(name)
flow = load_flow(name, None, lean=True)


print("Creating output file...")

init_file(name, cnfg, save, mesh, flow)


print("Reordering mesh data...")

mesh = sort_mesh(mesh, True)
flow = sort_flow(flow, mesh, lean=True)

u0_edge = flow.uu_edge[-1, :, 0]
uu_edge = u0_edge
ut_edge = u0_edge * 0.0

h0_cell = flow.hh_cell[-1, :, 0]
hh_cell = h0_cell
ht_cell = h0_cell * 0.0

hh_cell = np.maximum(HH_TINY, hh_cell)


print("Forming coefficients...")

# set sparse spatial operators
trsk = trsk_mats(mesh)

# remap fe,fc is more accurate?
flow.ff_edge = trsk.edge_stub_sums * flow.ff_vert
flow.ff_edge = \
    (flow.ff_edge / mesh.edge.area)

flow.ff_cell = trsk.cell_kite_sums * flow.ff_vert
flow.ff_cell = \
    (flow.ff_cell / mesh.cell.area)

flow.ff_cell *= (not cnfg.no_rotate)
flow.ff_edge *= (not cnfg.no_rotate)
flow.ff_vert *= (not cnfg.no_rotate)

kp_sums = np.zeros((
    cnfg.iteration // cnfg.stat_freq + 1), dtype=float)
en_sums = np.zeros((
    cnfg.iteration // cnfg.stat_freq + 1), dtype=float)


print('Done.')

Loading input assets...
Creating output file...
Reordering mesh data...
Forming coefficients...
Done.


Some preliminary performance tests.
The CSRmatrix-vector multiply is very fast -- let's compare to TT multiply.

Let's look at `trsk.edge_flux_perp`.
This has shape `(nedges, nedges)` and multiplies `uu_edge` (normal velocity at cell edges) to get the tangential velocity at cell edges, e.g.
```
vv_edge = trsk.edge_flux_perp @ uu_edge
```
For our performance tests, we will work with a random `uu_edge`.

In [4]:
#uu_edge = np.random.rand(mesh.edge.size)
uu_edge = np.ones(mesh.edge.size)
print(f'nedges = {mesh.edge.size}')

nedges = 30720


Before getting to the performance tests, let's build the TT/QTT operator and tensors we need and look at how expensive this is.

Since the number of edges here is not a multiple of 2, we can't trivially determine how to fold our tensors.
For now, we will just use a prime factorization of the number of edges.

For now, our operator matrix is square, so we will worry about how to fold a non-square matrix into a TT operator later...

In [5]:
prime_factors = list( factorint(mesh.edge.size).items() )
print(f'prime_factors = {prime_factors}')

tt_tens_shape = []
tt_op_shape = []
for factor in prime_factors:
    tt_tens_shape += [ factor[0] ] * factor[1]
    tt_op_shape += [ (factor[0], factor[0]) ] * factor[1]
# END for

print(f'tt_tens_shape = {tt_tens_shape}')
print(f'tt_op_shape = {tt_op_shape}')

prime_factors = [(2, 11), (3, 1), (5, 1)]
tt_tens_shape = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 5]
tt_op_shape = [(2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (3, 3), (5, 5)]


Now, let's form the TT operator and corresponding TT tensor, which will have shapes `tt_op_shape` and `tt_tens_shape` respectivly.

Notes:
1) The compression on the random `tt_uu_edge` is *horrible* (more than 2x) when we use `uu_edge = np.random.rand(mesh.edge.size)`. If the flow was more realistic, maybe this wouldn't be so bad? Should look into this later. This also causes the python kernel to die on my LANL mac when we go to do the TT multiply later. For now, we will test with `uu_edge = np.ones(mesh.edge.size)`.
3) It takes a long time to build `tt_edge_flux_perp`! With 30+ operators to build in this code, we could never do this at run time in practice. We would need to build/store once as associated with a given mesh.

In [6]:
tt_op_timer = Timer()
tt_round_op_timer = Timer()

tt_tens_timer = Timer()

# need to get dense version of the operator
# to pass to tt.TT()
dense_edge_flux_perp = trsk.edge_flux_perp.todense()


# form the tt tensor
tt_tens_timer.start()
tt_uu_edge = tt.TT(uu_edge, tt_tens_shape)
tt_tens_timer.stop()

print(f'tt_uu_edge = {tt_uu_edge}')
print(f'time to tt.TT(uu_edge) = {tt_tens_timer.get_time()}')


# form the tt operator
tt_op_timer.start()
tt_edge_flux_perp = tt.TT(dense_edge_flux_perp, tt_op_shape)
tt_op_timer.stop()

print(f'\ntt_edge_flux_perp = {tt_edge_flux_perp}')
print(f'time to tt.TT(dense_edge_flux_perp) = {tt_op_timer.get_time()}')

tt_uu_edge = TT with sizes and ranks:
N = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 5]
R = [1, np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), 1]

Device: cpu, dtype: torch.float64
#entries 30 compression 0.0009765625

time to tt.TT(uu_edge) = 0.0015568733215332031

tt_edge_flux_perp = TT-matrix with sizes and ranks:
M = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 5]
N = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 5]
R = [1, 4, np.int64(10), np.int64(22), np.int64(46), np.int64(94), np.int64(212), np.int64(628), np.int64(1464), np.int64(2570), 900, 225, 25, 1]
Device: cpu, dtype: torch.float64
#entries 29475394 compression 0.031233251359727647

time to tt.TT(dense_edge_flux_perp) = 137.49320816993713


The compression on the TT operator isnt bad, on the order of 1e-2.
However, we also need to look at the compression obtained by the CSR representation of the matrix.

In [7]:
csr_nstored = trsk.edge_flux_perp.nnz
print(f'csr_nstored = {csr_nstored}')
print(f'csr compression = {csr_nstored / mesh.edge.size**2}')

csr_nstored = 307140
csr compression = 0.0003254572550455729


CSR gives us a compression rate of 1e-4.
Two orders of magnitude better than TT.

Let's look at multiplication speeds.
First, at our TT multiply.
Since linear algebra operations on TTs cause the rank of the result to increase, we generally round after each `@` (note the awful compression before rounding).
We will time these two perations seperately, but understand that the total time is what is important.

In [8]:
tt_mult_timer = Timer()
tt_round_timer = Timer()

tt_mult_timer.start()
tt_mult_result = tt_edge_flux_perp @ tt_uu_edge
tt_mult_timer.stop()

print(f'tt_mult_result (before round) = {tt_mult_result}')
print(f'time to @ = {tt_mult_timer.get_time()}')


tt_round_timer.start()
tt_mult_result = tt_mult_result.round()
tt_round_timer.stop()

print(f'\ntt_mult_result (after round) = {tt_mult_result}')
print(f'time to round = {tt_round_timer.get_time()}')

print(f'\ntotal time = {tt_mult_timer.get_time() + tt_round_timer.get_time()}')

tt_mult_result (before round) = TT with sizes and ranks:
N = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 5]
R = [1, 4, 10, 22, 46, 94, 212, 628, 1464, 2570, 900, 225, 25, 1]

Device: cpu, dtype: torch.float64
#entries 14729072 compression 479.46197916666665

time to @ = 0.017527103424072266

tt_mult_result (after round) = TT with sizes and ranks:
N = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 5]
R = [1, 2, 4, 8, 16, 32, 64, 128, 120, 60, 30, 15, 5, 1]

Device: cpu, dtype: torch.float64
#entries 71714 compression 2.3344401041666667

time to round = 0.12090015411376953

total time = 0.2768545150756836


This seems pretty slow, let's look at the CSR multiply and the ratio of the two times.

In [9]:
csr_mult_timer = Timer()

csr_mult_timer.start()
csr_mult_result = trsk.edge_flux_perp @ uu_edge
csr_mult_timer.stop()

print(f'csr mult time = {csr_mult_timer.get_time()}')


print(f'tt mult time / csr mult time = {(tt_mult_timer.get_time() + tt_round_timer.get_time()) / csr_mult_timer.get_time()}')

csr mult time = 0.0005898475646972656
tt mult time / csr mult time = 352.0246564268391


CSR multiply is over 200x faster than the TT multiply + rounding.

The shape of the TT operator and the TT tensor could be important here, but it is hard to see how we are going to get 200x back. 
We could try padding the originall operator matrix and the state vector so that they have a size that is a multiple of 2 to get into true QTT format?

For now, let's see how long it would take to do the full matrix-vector multiply. 

In [10]:
full_mult_timer = Timer()

full_mult_timer.start()
full_mult_result = dense_edge_flux_perp @ uu_edge
full_mult_timer.stop()

print(f'full mult time = {full_mult_timer.get_time()}')

full mult time = 0.4231600761413574


This is sometimes faster than the TT multiply, sometimes slower.
Taking all this together, I would not be surprised to learn that I am doing something wrong here.

Finally, let's take a quick look at errors, taking `full_mult_result` as exact.

In [11]:
csr_err = np.sum( np.square(full_mult_result - csr_mult_result) )
print(f'csr_err = {csr_err}')

tt_err = np.sum( np.square(full_mult_result - tt_mult_result.full().flatten().numpy()) )
print(f'tt_err = {tt_err}')

csr_err = 1.1346079353288287e-28
tt_err = 7.690421473359942e-23
